In [6]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Flatten
from sklearn.preprocessing import MinMaxScaler as MMS, StandardScaler as SDS
from tensorflow.keras.callbacks import EarlyStopping 
import numpy as np

# 데이터

In [7]:
dataset = load_boston()        
x = dataset.data                
y = dataset.target              

# print("x: ", x, "\ny: ", y)
print("원본 데이터\n",x.shape, y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=3333)

# scaler = MMS()
scaler = SDS()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# print("x_test: ", x_test, "\nx_trian: ", x_train)
# print("y_test: ", y_test, "\ny_trian: ", y_train)
print("split + scailing x 데이터\n", x_train.shape, x_test.shape)
print("split + scailing y 데이터\n", y_train.shape, y_test.shape)

# ---------- RNN 모델에 적용하기위해 3차원으로 변환 ----------- #
x_train = x_train.reshape(-1, 13, 1)
x_test = x_test.reshape(-1, 13, 1)

원본 데이터
 (506, 13) (506,)
split + scailing x 데이터
 (354, 13) (152, 13)
split + scailing y 데이터
 (354,) (152,)


c:\Users\bitcamp\anaconda3\envs\tf27\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housin

In [8]:
model = Sequential()
model.add(LSTM(32, input_shape = (13, 1), return_sequences=True, activation='relu'))
model.add(Dropout(0.5)) # 과적합 방지
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3)) # 과적합 방지
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2)) # 과적합 방지
model.add(Dense(128, activation='relu'))
model.add(Flatten())
model.add(Dense(1))

# LSTM을 두 개의 layer에서 사용할 때에는 return_sequences 파라미터를 지정해야한다. (default = False)


# 컴파일, 훈련

In [9]:
model.compile(loss = 'mse', optimizer='adam')

earlyStopping = EarlyStopping(monitor='val_loss', mode='min', patience=16, restore_best_weights=True, verbose=3) # loss - min, accuracy - max 
hist = model.fit(x_train, y_train, epochs=256, batch_size=16, validation_split=0.2, callbacks = [earlyStopping], verbose=3) # verbose: 함수 수행시 발생하는 상세한 정보들을 표준 출력으로 자세히 내보낼 것인지

Epoch 1/256
Epoch 2/256
Epoch 3/256
Epoch 4/256
Epoch 5/256
Epoch 6/256
Epoch 7/256
Epoch 8/256
Epoch 9/256
Epoch 10/256
Epoch 11/256
Epoch 12/256
Epoch 13/256
Epoch 14/256
Epoch 15/256
Epoch 16/256
Epoch 17/256
Epoch 18/256
Epoch 19/256
Epoch 20/256
Epoch 21/256
Epoch 22/256
Epoch 23/256
Epoch 24/256
Epoch 25/256
Epoch 26/256
Epoch 27/256
Epoch 28/256
Epoch 29/256
Epoch 30/256
Epoch 31/256
Epoch 32/256
Epoch 33/256
Epoch 34/256
Epoch 35/256
Epoch 36/256
Epoch 37/256
Epoch 38/256
Epoch 39/256
Epoch 40/256
Epoch 41/256
Restoring model weights from the end of the best epoch: 25.
Epoch 00041: early stopping


# 평가, 예측

In [10]:
loss = model.evaluate(x_test, y_test, verbose=3)
print('loss(mse): ', loss)
x_test

y_predict = model.predict(x_test)
# print('x_test:\n', x_test)
# print('y_predict:\n', y_predict)

# print(hist) # <keras.callbacks.History object at 0x000001ECB4986D00>
# print(hist.history) # 딕셔너리(key, value) → loss의 변화값을 list로(value는 list로 저장된다.)  

RMSE = np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE: ", RMSE)

r2 = r2_score(y_test, y_predict)
print("R2: ", r2)

loss(mse):  19.062942504882812
RMSE:  4.366113192982679
R2:  0.7353560207210108
